## What is DataWallet? 

![alt text](https://i2.wp.com/coinfunda.com/wp-content/uploads/2017/12/DataWallet-App.png?ssl=1 "Logo Title Text 1")

- A blockchain-based data-exchange that allows users to earn a passive income from their data
- Users get rewarded either via the proprietary network token (DXT) or via personalized services
- It gives users complete ownership over their own data, while giving developers and data consumers access to a marketplace for high-quality data. 
- Data consumers use data exchange tokens (DXT) to purchase data directly from creators using a smart contract which ensures security and transparency from both parties. 
- It also provides a data API to enables developers to use this data to build apps.

## Why is it Useful? 

![alt text](https://lh4.googleusercontent.com/oDnoAiArLVDCG8IFnnPgoNxZE153UpGRfkVJGAfW6lAXkmoVeG1opv_8AODMKn0IHRpt-B6ZrbLYfWizTCSP0oignkOcwzT1KnvCG7QClG7D4bwwGW-jRpaLfv4vX3FIzeVBh4bE=w1200-h630-p-k-no-nu "Logo Title Text 1")

- 90% of the world's data has been generated in the past 2 years. Thats a lot of data! We need a new business model
- Data generated through your use of social media and other online platforms are currently controlled by those companies
- DataWallet lets you share data as you wish and earn money from it.
- A Datawallet profile gives you data mobility, allowing you to easily plug into any AI-powered app and get personalized services.
- It allows you to trade anonymized parts of your data to businesses or developers who list contract offers. 
- Contracts specify the type of data it will collect and the benefit offered, namely Datawallet’s data exchange token DXT. Datawallet will provide payout options
- User data production is fragmented across different platforms, Data Brokers must employ probabilistic models to approximate the likelihood of data points belonging to a given user. These highly fallible models lead to misunderstanding of target audiences.

## Applications

![alt text](https://image.slidesharecdn.com/marketingresearchch5malhotra-140725014700-phpapp02/95/marketing-research-ch-5malhotra-6-638.jpg?cb=1406252902 "Logo Title Text 1")

- Market Research- It allows for entirely automated data capture which eradicates the biases associated with traditional surveying techniques. 
- Deterministic Advertising-  Advertisers can deterministically find the right target customers based on verifiable historic as well as real-time social media and purchase data. 
- Personalized Car Insurance - A user’s collated data wallet is exactly what Auto insurers are looking for
- Personalized Loan Underwriting - lenders are always searching for ways to more accurately determine a borrower’s probability of defaulting.
- Psychographic Hiring - A digitally interpretable collation of an applicant’s experiences, talents, and personality (their data)

## Tech Stack

![alt text](https://i0.wp.com/globalhalo.com/wp-content/uploads/2018/01/datawallet.jpg?resize=720%2C368&ssl=1 "Logo Title Text 1")

![alt text](https://www.2ndquadrant.com/media/filer_public_thumbnails/filer_public/a8/54/a854c5d1-82cd-47a8-83bf-6838aa6d762d/postgres-xl_display.png__650x465_q85_subsampling-2.png "Logo Title Text 1")

Databases:
- API usage: POSTGRESQL + REDIS
- Pipeline usage: AURORA
Languages:
- Front-end: Javascript ES2018 via babel. Swift 4, Java8
- Back-end: Node LTE, Python 3, Golang
Infrastructure:
- Front-end: Cloudfront + S3
- Back-end: Scaling groups + EC2 + ELB
- Monitoring: Cloudwatch
- Deployment: Terraform
- Blockchain: Ethereum 

![alt text](https://cdn-images-1.medium.com/max/1600/0*B_K9HNktRm_DrEZT.png "Logo Title Text 1")


## Solidity Example of exchange contract 

```
pragma solidity ^0.4.21;

contract Purchase {
    uint public value;
    address public seller;
    address public buyer;
    enum State { Created, Locked, Inactive }
    State public state;

    // Ensure that `msg.value` is an even number.
    // Division will truncate if it is an odd number.
    // Check via multiplication that it wasn't an odd number.
    function Purchase() public payable {
        seller = msg.sender;
        value = msg.value / 2;
        require((2 * value) == msg.value);
    }

    modifier condition(bool _condition) {
        require(_condition);
        _;
    }

    modifier onlyBuyer() {
        require(msg.sender == buyer);
        _;
    }

    modifier onlySeller() {
        require(msg.sender == seller);
        _;
    }

    modifier inState(State _state) {
        require(state == _state);
        _;
    }

    event Aborted();
    event PurchaseConfirmed();
    event ItemReceived();

    /// Abort the purchase and reclaim the ether.
    /// Can only be called by the seller before
    /// the contract is locked.
    function abort()
        public
        onlySeller
        inState(State.Created)
    {
        emit Aborted();
        state = State.Inactive;
        seller.transfer(this.balance);
    }

    /// Confirm the purchase as buyer.
    /// Transaction has to include `2 * value` ether.
    /// The ether will be locked until confirmReceived
    /// is called.
    function confirmPurchase()
        public
        inState(State.Created)
        condition(msg.value == (2 * value))
        payable
    {
        emit PurchaseConfirmed();
        buyer = msg.sender;
        state = State.Locked;
    }

    /// Confirm that you (the buyer) received the item.
    /// This will release the locked ether.
    function confirmReceived()
        public
        onlyBuyer
        inState(State.Locked)
    {
        emit ItemReceived();
        // It is important to change the state first because
        // otherwise, the contracts called using `send` below
        // can call in again here.
        state = State.Inactive;

        // NOTE: This actually allows both the buyer and the seller to
        // block the refund - the withdraw pattern should be used.

        buyer.transfer(value);
        seller.transfer(this.balance);
    }
}
```